In [39]:
def default_preprocessing(df):
    """Custom preprocessing function to prepare MEPS data, focusing on specified features."""
    def race(row):
        if row['HISPANX'] == 2 and row['RACEV2X'] == 1:
            return 'White'
        return 'Non-White'
    def race2(row):
        if row['HISPANX'] == 2 and row['RACEV2X'] == 1:
            return 'White'
        elif row['RACEV2X'] == 2:
            return 'Black'
        elif row['RACEV2X'] == 3:
            return 'American Indian'
        elif row['RACEV2X'] == 4:
            return 'Asian Indian'
        elif row['RACEV2X'] == 5:
            return 'Chinese'
        elif row['RACEV2X'] == 6:
            return 'Filipino'
        elif row['RACEV2X'] == 10:
            return 'Other'
        elif row['RACEV2X'] == 12:
            return 'Multiplle'
        else:
            return 'NA'
    # Apply transformations
    df['RACE'] = df.apply(race, axis=1)
    df['RACE_EXP'] = df.apply(race2, axis=1)

    df = df[df['PANEL'] == 19]  # Restrict to Panel 19

    # Rename columns as necessary
    rename_dict = {
        'FTSTU53X': 'FTSTU', 'ACTDTY53': 'ACTDTY', 'HONRDC53': 'HONRDC',
        'RTHLTH53': 'RTHLTH', 'MNHLTH53': 'MNHLTH', 'CHBRON53': 'CHBRON', 
        'JTPAIN53': 'JTPAIN', 'PREGNT53': 'PREGNT', 'WLKLIM53': 'WLKLIM', 
        'ACTLIM53': 'ACTLIM', 'SOCLIM53': 'SOCLIM', 'COGLIM53': 'COGLIM', 
        'EMPST53': 'EMPST', 'REGION53': 'REGION', 'MARRY53X': 'MARRY', 
        'AGE53X': 'AGE', 'POVCAT15': 'POVCAT', 'INSCOV15': 'INSCOV'
    }
    df.rename(columns=rename_dict, inplace=True)

    # Calculate UTILIZATION
    def utilization(row):
        return row['OBTOTV15'] + row['OPTOTV15'] + row['ERTOT15'] + row['IPNGTD15'] + row['HHTOTD15']

    df['UTILIZATION'] = df.apply(utilization, axis=1)
    df['UTILIZATION'] = (df['UTILIZATION'] >= 10).astype(float)

    # Filter rows based on valid values
    valid_criteria = {
        'REGION': 0, 'AGE': 0, 'MARRY': 0, 'ASTHDX': 0,
        'FTSTU': -1, 'ACTDTY': -1, 'HONRDC': -1, 'RTHLTH': -1, 'MNHLTH': -1,
        'HIBPDX': -1, 'CHDDX': -1, 'ANGIDX': -1, 'EDUCYR': -1, 'HIDEG': -1,
        'MIDX': -1, 'OHRTDX': -1, 'STRKDX': -1, 'EMPHDX': -1, 'CHBRON': -1,
        'CHOLDX': -1, 'CANCERDX': -1, 'DIABDX': -1, 'JTPAIN': -1, 'ARTHDX': -1,
        'ARTHTYPE': -1, 'ASTHDX': -1, 'ADHDADDX': -1, 'PREGNT': -1, 'WLKLIM': -1,
        'ACTLIM': -1, 'SOCLIM': -1, 'COGLIM': -1, 'DFHEAR42': -1, 'DFSEE42': -1,
        'ADSMOK42': -1, 'PHQ242': -1, 'EMPST': -1, 'POVCAT': -1, 'INSCOV': -1
    }

    for column, cutoff in valid_criteria.items():
        df = df[df[column] >= cutoff]

    # Specify the features to keep
    features_to_keep = [
        'REGION', 'AGE', 'SEX', 'RACE', 'MARRY', 'FTSTU', 'ACTDTY', 'HONRDC', 'RTHLTH', 'MNHLTH',
        'HIBPDX', 'CHDDX', 'ANGIDX', 'MIDX', 'OHRTDX', 'STRKDX', 'EMPHDX', 'CHBRON', 'CHOLDX',
        'CANCERDX', 'DIABDX', 'JTPAIN', 'ARTHDX', 'ARTHTYPE', 'ASTHDX', 'ADHDADDX', 'PREGNT',
        'WLKLIM', 'ACTLIM', 'SOCLIM', 'COGLIM', 'DFHEAR42', 'DFSEE42', 'ADSMOK42', 'PCS42',
        'MCS42', 'K6SUM42', 'PHQ242', 'EMPST', 'POVCAT', 'INSCOV', 'UTILIZATION', 'PERWT15F','RACE_EXP'
    ]

    # Drop all other features not listed
    features_to_drop = [col for col in df.columns if col not in features_to_keep]
    df.drop(columns=features_to_drop, inplace=True)

    return df


In [40]:
filepath = 'h181.csv'
df = pd.read_csv(filepath)
df_transformed = default_preprocessing(df)
df_transformed.head()


C:\Users\srinivas\AppData\Local\Temp\ipykernel_30680\2599504375.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_dict, inplace=True)
C:\Users\srinivas\AppData\Local\Temp\ipykernel_30680\2599504375.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UTILIZATION'] = df.apply(utilization, axis=1)
C:\Users\srinivas\AppData\Local\Temp\ipykernel_30680\2599504375.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,REGION,AGE,SEX,MARRY,FTSTU,ACTDTY,HONRDC,RTHLTH,MNHLTH,HIBPDX,...,MCS42,K6SUM42,PHQ242,EMPST,POVCAT,INSCOV,PERWT15F,RACE,RACE_EXP,UTILIZATION
0,2,53,1,5,-1,2,2,4,3,1,...,58.47,3,0,4,1,2,21854.981705,White,White,1.0
1,2,56,2,3,-1,2,2,4,3,1,...,26.57,17,6,4,3,2,18169.604822,White,White,1.0
3,2,23,2,5,3,2,2,1,1,2,...,50.33,7,0,1,2,2,17191.832515,White,White,0.0
4,2,3,1,6,-1,3,3,1,3,-1,...,-1.00,-1,-1,-1,2,2,20261.485463,White,White,0.0
5,3,27,1,1,-1,1,4,2,1,2,...,-1.00,-1,-1,1,3,1,0.000000,Non-White,Multiplle,0.0


In [42]:
df_transformed.columns

Index(['REGION', 'AGE', 'SEX', 'MARRY', 'FTSTU', 'ACTDTY', 'HONRDC', 'RTHLTH',
       'MNHLTH', 'HIBPDX', 'CHDDX', 'ANGIDX', 'MIDX', 'OHRTDX', 'STRKDX',
       'EMPHDX', 'CHBRON', 'CHOLDX', 'CANCERDX', 'DIABDX', 'JTPAIN', 'ARTHDX',
       'ARTHTYPE', 'ASTHDX', 'ADHDADDX', 'PREGNT', 'WLKLIM', 'ACTLIM',
       'SOCLIM', 'COGLIM', 'DFHEAR42', 'DFSEE42', 'ADSMOK42', 'PCS42', 'MCS42',
       'K6SUM42', 'PHQ242', 'EMPST', 'POVCAT', 'INSCOV', 'PERWT15F', 'RACE',
       'RACE_EXP', 'UTILIZATION'],
      dtype='object')

In [43]:
len(df_transformed)

15830

In [44]:
df_transformed.to_csv('MEPS_FINAL.csv',index=False)

In [48]:
df_transformed.select_dtypes(include=[float]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 15830 entries, 0 to 16577
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PCS42        15830 non-null  float64
 1   MCS42        15830 non-null  float64
 2   PERWT15F     15830 non-null  float64
 3   UTILIZATION  15830 non-null  float64
dtypes: float64(4)
memory usage: 618.4 KB
